## Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, BayesianRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn import svm

from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime

DATEPARSE = "%Y-%m-%d %H:%M:%S"

# Read data
### Add data into /taxi_data/
### Github stores a sample file of this. Download the rest and do the same.

In [2]:
def add_zero_padding(number):
    if len(str(number)) == 1: return "0" + str(number)
    else: return number
def format_df(DF, pickup_date_column, dropof_date_column):
    for i, x in DF.iterrows():
        DF.at[i,"month"] = 1
        start = datetime.strptime(x[pickup_date_column], DATEPARSE).timestamp()
        stop = datetime.strptime(x[dropof_date_column], DATEPARSE).timestamp()
        DF.at[i, pickup_date_column] = start
        DF.at[i, dropof_date_column] = stop
        DF.at[i, "travel_time"] = stop - start
    return DF

In [3]:
#GREEN
header = True
for x in range(1, 13): # this takes 12 min
    try:
        dfGreen = pd.read_csv("taxi_data\green_tripdata_2019-" + str(add_zero_padding(x)) + ".csv", usecols=["PULocationID", "DOLocationID", "lpep_pickup_datetime", "lpep_dropoff_datetime"])
        dfGreen = format_df(dfGreen, "lpep_pickup_datetime", "lpep_dropoff_datetime")
        dfGreen.to_csv("taxi_data\green_tripdata_2019_concat.csv", mode='a+', header=header, index=False)
        header=False
        print("GREEN: " + str(x))
    except Exception as f:
        print(f)

"""
# YELLOW
header = True
for x in range(1, 13):
    try:
        dfYellow = pd.read_csv("taxi_data\yellow_tripdata_2019-" + str(add_zero_padding(x)) + ".csv")
        dfYellow = format_df(dfYellow, "tpep_pickup_datetime", "tpep_dropoff_datetime")
        dfYellow.to_csv("taxi_data\yellow_tripdata_2019_concat.csv", mode='a+', header=header, index=False)
        header=False
        print("YELLOW: " + str(x))
    except Exception as f:
        print(f)
"""

GREEN: 1
GREEN: 2
[Errno 2] No such file or directory: 'taxi_data\\green_tripdata_2019-03.csv'
GREEN: 4
GREEN: 5
GREEN: 6
GREEN: 7


# Train

In [2]:
dfGreen = pd.read_csv("taxi_data\green_tripdata_2019_concat.csv", chunksize=1000000)

model = svm.SVC()

first = True
for chunk in dfGreen:
    del dfGreen
    X_green = chunk[["PULocationID", "DOLocationID","lpep_pickup_datetime"]] # input cols
    y_green = chunk[["travel_time"]] # output cols

    X_green = np.array(X_green)
    y_green = np.array(y_green)

    X_train, X_test, y_train, y_test = train_test_split(X_green, y_green, test_size = 0.25)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
    print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))
    
    break

C:\Python310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
